In [ ]:
import os
import logging
import tkinter as tk
from tkinter import filedialog, messagebox
import json
import pandas as pd
import numpy as np
from prophet import Prophet
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from datetime import timedelta

# GUI-Warnungen unterdrücken
os.environ['TK_SILENCE_DEPRECATION'] = '1'

# cmdstanpy-Protokollierung auf WARN setzen
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# Standard-Konfigurationsdatei
CONFIG_FILE = "config.json"

# Standardkonfiguration
DEFAULT_CONFIG = {
    "forecast_horizon": 30,  # Forecast-Horizont in Tagen
    "api_keys": {
        "openweathermap": "Ihr_API_Schlüssel",
        "google_maps": "Ihr_API_Schlüssel",
        "ticketmaster": "Ihr_API_Schlüssel"
    },
    "location": {
        "latitude": 52.5200,
        "longitude": 13.4050
    }
}

# Konfigurationsdatei laden
def load_config():
    if not os.path.exists(CONFIG_FILE):
        save_config(DEFAULT_CONFIG)
    with open(CONFIG_FILE, "r") as f:
        return json.load(f)

# Konfigurationsdatei speichern
def save_config(config):
    with open(CONFIG_FILE, "w") as f:
        json.dump(config, f, indent=4)

# Wetterdaten abrufen
def fetch_weather_data(start_date, end_date, lat, lon, api_key):
    weather_data = []
    current_date = start_date
    while current_date <= end_date:
        timestamp = int(current_date.timestamp())
        url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine"
        response = requests.get(url, params={
            "lat": lat,
            "lon": lon,
            "dt": timestamp,
            "appid": api_key,
            "units": "metric"
        })
        if response.status_code == 200:
            data = response.json()
            weather_data.append({
                "ds": current_date,
                "temperature": data['current']['temp'],
                "humidity": data['current']['humidity']
            })
        current_date += timedelta(days=1)
    return pd.DataFrame(weather_data)

# Verkehrsdaten (simuliert)
def fetch_traffic_data(start_date, end_date):
    traffic_data = []
    for single_date in pd.date_range(start=start_date, end=end_date):
        traffic_intensity = np.random.randint(0, 10)
        traffic_data.append({
            "ds": single_date,
            "traffic_intensity": traffic_intensity
        })
    return pd.DataFrame(traffic_data)

# Eventdaten (simuliert)
def fetch_event_data(start_date, end_date):
    event_data = []
    for single_date in pd.date_range(start=start_date, end=end_date):
        event_count = np.random.randint(0, 5)
        event_data.append({
            "ds": single_date,
            "event_count": event_count
        })
    return pd.DataFrame(event_data)

# Haupt-GUI
class ForecastApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Forecasting Tool")
        self.root.geometry("1200x800")
        self.config = load_config()

        # Startmenü
        self.setup_main_menu()

    def setup_main_menu(self):
        main_menu_frame = tk.Frame(self.root)
        main_menu_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        tk.Label(main_menu_frame, text="Willkommen im Forecasting Tool", font=("Arial", 16)).pack(pady=20)

        tk.Button(main_menu_frame, text="Einstellungen", command=self.open_settings, width=20).pack(pady=10)
        tk.Button(main_menu_frame, text="Forecast erstellen", command=self.open_forecast_tool, width=20).pack(pady=10)

    def open_settings(self):
        settings_window = tk.Toplevel(self.root)
        settings_window.title("Einstellungen")
        settings_window.geometry("600x400")

        tk.Label(settings_window, text="Einstellungen bearbeiten", font=("Arial", 14)).pack(pady=10)

        # Forecast-Horizont
        tk.Label(settings_window, text="Forecast-Horizont (Tage):").pack(anchor="w", padx=10)
        horizon_entry = tk.Entry(settings_window)
        horizon_entry.insert(0, self.config["forecast_horizon"])
        horizon_entry.pack(fill=tk.X, padx=10, pady=5)

        # OpenWeatherMap API Key
        tk.Label(settings_window, text="OpenWeatherMap API Key:").pack(anchor="w", padx=10)
        owm_entry = tk.Entry(settings_window)
        owm_entry.insert(0, self.config["api_keys"]["openweathermap"])
        owm_entry.pack(fill=tk.X, padx=10, pady=5)

        # Speichern und Schließen
        def save_settings():
            self.config["forecast_horizon"] = int(horizon_entry.get())
            self.config["api_keys"]["openweathermap"] = owm_entry.get()
            save_config(self.config)
            messagebox.showinfo("Erfolg", "Einstellungen wurden gespeichert.")
            settings_window.destroy()

        tk.Button(settings_window, text="Speichern", command=save_settings).pack(pady=10)

    def open_forecast_tool(self):
        ForecastTool(self.root, self.config)

# Forecast-Tool GUI
class ForecastTool:
    def __init__(self, root, config):
        self.config = config
        self.data = None
        self.forecast = None

        forecast_window = tk.Toplevel(root)
        forecast_window.title("Forecast erstellen")
        forecast_window.geometry("1200x800")

        # GUI-Elemente
        button_frame = tk.Frame(forecast_window)
        button_frame.pack(fill=tk.X, pady=5)

        load_data_button = tk.Button(button_frame, text="Daten laden", command=self.load_data)
        load_data_button.pack(side=tk.LEFT, padx=5)

        meta_data_button = tk.Button(button_frame, text="Meta-Daten hinzufügen", command=self.add_meta_data)
        meta_data_button.pack(side=tk.LEFT, padx=5)

        start_forecast_button = tk.Button(button_frame, text="Forecast starten", command=self.start_forecast)
        start_forecast_button.pack(side=tk.LEFT, padx=5)

        save_button = tk.Button(button_frame, text="Ergebnisse speichern", command=self.save_results)
        save_button.pack(side=tk.LEFT, padx=5)

        self.plot_frame = tk.Frame(forecast_window)
        self.plot_frame.pack(fill=tk.BOTH, expand=True)

    def load_data(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV-Dateien", "*.csv")])
        if file_path:
            self.data = pd.read_csv(file_path)
            self.data['ds'] = pd.to_datetime(self.data['ds'])
            messagebox.showinfo("Erfolg", f"Daten erfolgreich geladen: {len(self.data)} Zeilen.")
        else:
            messagebox.showwarning("Warnung", "Keine Datei ausgewählt.")

    def add_meta_data(self):
        try:
            if self.data is None:
                raise ValueError("Bitte laden Sie zuerst Zeitreihendaten.")
            start_date = self.data['ds'].min()
            end_date = self.data['ds'].max()
            weather = fetch_weather_data(start_date, end_date, self.config["location"]["latitude"], self.config["location"]["longitude"], self.config["api_keys"]["openweathermap"])
            traffic = fetch_traffic_data(start_date, end_date)
            events = fetch_event_data(start_date, end_date)
            meta_data = weather.merge(traffic, on="ds", how="outer").merge(events, on="ds", how="outer")
            self.data = self.data.merge(meta_data, on="ds", how="left")
            messagebox.showinfo("Erfolg", "Meta-Daten erfolgreich hinzugefügt.")
        except Exception as e:
            messagebox.showerror("Fehler", f"Fehler beim Hinzufügen von Meta-Daten: {e}")

    def start_forecast(self):
        try:
            forecast_days = self.config["forecast_horizon"]
            model = Prophet()
            for col in ['temperature', 'humidity', 'traffic_intensity', 'event_count']:
                if col in self.data.columns:
                    model.add_regressor(col)
            model.fit(self.data)
            future = model.make_future_dataframe(periods=forecast_days)
            for col in ['temperature', 'humidity', 'traffic_intensity', 'event_count']:
                if col in self.data.columns:
                    future[col] = self.data[col].iloc[-forecast_days:].values
            self.forecast = model.predict(future)
            self.plot_forecast()
        except Exception as e:
            messagebox.showerror("Fehler", f"Fehler beim Forecast: {e}")

    def plot_forecast(self):
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(self.forecast['ds'], self.forecast['yhat'], label="Vorhersage")
        if 'y' in self.data.columns:
            ax.plot(self.data['ds'], self.data['y'], label="Tatsächliche Werte")
        ax.fill_between(self.forecast['ds'], self.forecast['yhat_lower'], self.forecast['yhat_upper'], alpha=0.3, label="Unsicherheitsintervall")
        ax.legend()
        ax.set_title("Forecast mit Meta-Daten")
        ax.set_xlabel("Datum")
        ax.set_ylabel("Wert")
        for widget in self.plot_frame.winfo_children():
            widget.destroy()
        canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    def save_results(self):
        save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV-Dateien", "*.csv")])
        if save_path:
            self.forecast.to_csv(save_path, index=False)
            messagebox.showinfo("Erfolg", "Ergebnisse erfolgreich gespeichert.")

# Hauptprogramm
if __name__ == "__main__":
    root = tk.Tk()
    app = ForecastApp(root)
    root.mainloop()